In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import pandas as pd
%cd "/content/gdrive/My Drive/thesis/Data"

#csv import labels

train_ACL_labels = np.array(pd.read_csv("train-acl.csv", header=None).iloc[:,1])
train_abnormal_labels = np.array(pd.read_csv("train-abnormal.csv", header=None).iloc[:,1])
train_meniscus_labels = np.array(pd.read_csv("train-meniscus.csv", header=None).iloc[:,1])

valid_ACL_labels = np.array(pd.read_csv("valid-acl.csv", header=None).iloc[:,1])
valid_abnormal_labels = np.array(pd.read_csv("valid-abnormal.csv", header=None).iloc[:,1])
valid_meniscus_labels = np.array(pd.read_csv("valid-meniscus.csv", header=None).iloc[:,1])

test_ACL_labels = np.array(pd.read_csv("test-acl.csv", header=None).iloc[:,1])
test_abnormal_labels = np.array(pd.read_csv("test-abnormal.csv", header=None).iloc[:,1])
test_meniscus_labels = np.array(pd.read_csv("test-meniscus.csv", header=None).iloc[:,1])

#data path
train_path = "/content/gdrive/My Drive/thesis/Data/train"
train_axial_path = "/content/gdrive/My Drive/thesis/Data/train/coronal"

counter = 5
for filename in os.listdir(train_axial_path):
  if counter > 0:
    file0 = np.load(train_axial_path + '/' + filename)
    variancelist = []
    for slice in range(file0.shape[0]):
      variancelist.append(np.var(file0[slice,:,:]))
    #print(file0)
    #print('max', np.amax(file0))
    #print('mean', np.mean(file0))
    #print(filename, 'file shape', file0.shape, 'max_var_index', variancelist.index(max(variancelist)), max(variancelist))
    counter = counter - 1

print(valid_ACL_labels)
print("done!")

/content/gdrive/My Drive/thesis/Data
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 1 0 1 0]
done!


In [0]:
# model.py

import torch
import torch.nn as nn

from torchvision import models

class MRNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.alexnet(pretrained=False)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, 1)

    # change this to adapt to different networks
    def forward(self, x):
        x = torch.squeeze(x, dim=0) # only batch size 1 supported
        x = self.model.features(x)
        # make sure that gap returns size 256
        x = self.gap(x).view(x.size(0), -1)
        x = torch.max(x, 0, keepdim=True)[0]
        x = self.classifier(x)
        return x

In [0]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

class PlainRAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
                    
        self.degenerated_to_sgd = degenerated_to_sgd
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)

        super(PlainRAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(PlainRAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                beta2_t = beta2 ** state['step']
                N_sma_max = 2 / (1 - beta2) - 1
                N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)


                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif self.degenerated_to_sgd:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    step_size = group['lr'] / (1 - beta1 ** state['step'])
                    p_data_fp32.add_(-step_size, exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss


class AdamW(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, warmup = 0):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, warmup = warmup)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                
                if group['warmup'] > state['step']:
                    scheduled_lr = 1e-8 + state['step'] * group['lr'] / group['warmup']
                else:
                    scheduled_lr = group['lr']

                step_size = scheduled_lr * math.sqrt(bias_correction2) / bias_correction1
                
                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * scheduled_lr, p_data_fp32)

                p_data_fp32.addcdiv_(-step_size, exp_avg, denom)

                p.data.copy_(p_data_fp32)

        return loss

In [0]:
import argparse
import json
import numpy as np
import os
import torch
from datetime import datetime
from pathlib import Path
from sklearn import metrics

def train(rundir, diagnosis, orientation, epochs, learning_rate, transformbool, use_gpu):
    
    val_auc_array = list()
    train_auc_array = list()
    test_auc_array = list()
    train_loader, valid_loader, test_loader = load_data(diagnosis, orientation, transformbool, use_gpu)
    
    model = MRNet()

    if use_gpu:
        model = model.cuda()

    optimizer = RAdam(model.parameters(), learning_rate, weight_decay=.01)

    # patience too low (after 5 epochs, if AUC hasnt improved, slash learning rate .3), which is why high learning rate seems to work better
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=.3, threshold=1e-4)

    best_val_loss = float('inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer)
        #print(f'train loss: {train_loss:0.4f}')
        #print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader)
        #print(f'valid loss: {val_loss:0.4f}')
        #print(f'valid AUC: {val_auc:0.4f}')

        test_loss, test_auc, _, _ = run_model(model, test_loader)

        val_auc_array.append(val_auc)
        train_auc_array.append(train_auc)
        test_auc_array.append(test_auc)
        
        scheduler.step(val_loss)
  
    
    file_name = f'val{val_auc:0.4f}_train{train_auc:0.4f}_test{test_auc:0.4f}_epoch{epoch+1}'
    save_path = "/content/gdrive/My Drive/thesis/Results/models" + '/' + str(diagnosis) + '/' + str(orientation) + "/" + file_name
    torch.save(model.state_dict(), save_path)
    print('model saved at', str(save_path))
        
    return val_auc_array, train_auc_array, test_auc_array

In [0]:
# evaluate.py

import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch

from sklearn import metrics
from torch.autograd import Variable

#from loader import load_data
#from model import MRNet

def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str, required=True)
    parser.add_argument('--split', type=str, required=True)
    parser.add_argument('--diagnosis', type=int, required=True)
    parser.add_argument('--gpu', action='store_true')
    return parser

def run_model(model, loader, train=False, optimizer=None):
    preds = []
    labels = []

    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.
    num_batches = 0

    for batch in loader:
        if train:
            optimizer.zero_grad()

        vol, label = batch
        if loader.dataset.use_gpu:
            vol = vol.cuda()
            label = label.cuda()
        vol = Variable(vol)
        label = Variable(label)

        logit = model.forward(vol)

        loss = loader.dataset.weighted_loss(logit, label)
        total_loss += loss.item()

        #
        pred = torch.sigmoid(logit)
        pred_npy = pred.data.cpu().numpy()[0][0]
        label_npy = label.data.cpu().numpy()[0][0]

        preds.append(pred_npy)
        labels.append(label_npy)

        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1

    avg_loss = total_loss / num_batches

    fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    return avg_loss, auc, preds, labels

def evaluate(split, model_path, diagnosis, orientation, use_gpu):
    train_loader, valid_loader, test_loader = load_data(diagnosis, orientation, transformbool, use_gpu)
    model = MRNet()
    state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
    model.load_state_dict(state_dict)

    if use_gpu:
        model = model.cuda()

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")

    loss, auc, preds, labels = run_model(model, loader)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    return preds, labels

#if __name__ == '__main__':
#    args = get_parser().parse_args()
#   evaluate(args.split, args.model_path, args.diagnosis, args.gpu)

Change Transforms


In [7]:
# loader.py

!pip install medicaltorch

import numpy as np
import os
import pickle
import torch
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from medicaltorch import transforms as mt_transforms
import PIL
from random import sample

from torch.autograd import Variable

INPUT_DIM = 224
MAX_PIXEL_VAL = 255
#MEAN = 58.09
#STDDEV = 49.73

class Dataset(data.Dataset):
    def __init__(self, datadirs, diagnosis, orientation, use_gpu, transformbool):
        super().__init__()
        self.use_gpu = use_gpu
        self.transformbool = transformbool
        label_dict = {}
        self.paths = []
        print(datadirs)
        
        self.orientation = orientation
        self.diagnosis = diagnosis

        train_string = "/content/gdrive/My Drive/thesis/Data/train"
        valid_string = "/content/gdrive/My Drive/thesis/Data/valid"
        test_string = "/content/gdrive/My Drive/thesis/Data/test"

        if datadirs == train_string:
          if diagnosis == 'ACL':
            self.labels = train_ACL_labels
          if diagnosis == 'meniscus':
            self.labels = train_meniscus_labels
          if diagnosis == 'abnormal':
            self.labels = train_abnormal_labels
        if datadirs == valid_string:
          if diagnosis == 'ACL':
            self.labels = valid_ACL_labels
          if diagnosis == 'meniscus':
            self.labels = valid_meniscus_labels
          if diagnosis == 'abnormal':
            self.labels = valid_abnormal_labels
        if datadirs == test_string:
          if diagnosis == 'ACL':
            self.labels = test_ACL_labels
          if diagnosis == 'meniscus':
            self.labels = test_meniscus_labels
          if diagnosis == 'abnormal':
            self.labels = test_abnormal_labels

        direct = datadirs + '/' + self.orientation
        for file in os.listdir(direct):
          self.paths.append(direct + '/' + file)
        self.paths.sort()

        #print("paths", self.paths[0:10])

        neg_weight = np.mean(self.labels)
        self.weights = [neg_weight, 1 - neg_weight]

        print(self.labels.shape)
        print(self.weights)

    def weighted_loss(self, prediction, target):
        weights_npy = np.array([self.weights[int(t[0])] for t in target.data])
        weights_tensor = torch.FloatTensor(weights_npy)
        if self.use_gpu:
            weights_tensor = weights_tensor.cuda()
        loss = F.binary_cross_entropy_with_logits(prediction, target, weight=Variable(weights_tensor))
        return loss

    # Data augmentation section
    # can go through each cases, looking at the histogram of 3T vs 1.5T (naive distribution of contrast data?)
    def __getitem__(self, index):
        path = self.paths[index]
        
        vol = np.load(path)

        
        ax_mean = 63.16
        ax_std = 60.46
        cor_mean = 59.27
        cor_std = 64.00
        sag_mean = 58.25
        sag_std = 48.15
        

        # standardize
        vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol) + 1.0e-6) * MAX_PIXEL_VAL
        
        
        if self.orientation == 'axial':
          MEAN = ax_mean
          STDDEV = ax_std
        if self.orientation == 'coronal':
          MEAN = cor_mean
          STDDEV = cor_std
        if self.orientation == 'sagittal':
          MEAN = sag_mean
          STDDEV = sag_std
        

        vol = (vol - MEAN) / STDDEV

        vol = vol.astype(np.float32)

        flag = False
        randomangle = 0
        trans = 0

        # define transform policy
        hor_flip = np.random.rand(1)
        ran_rot = np.random.rand(1)
        randomangle = np.random.uniform(-20, 20)
        uni_noise = np.random.rand(1)
        ran_trans = np.random.rand(1)
        persp = np.random.rand(1)
        rand_erase = np.random.rand(1)

        if ran_trans < 0.5:
          trans = 0.05
        if ran_rot < 0.5:
          randomangle = 0
        
        if self.transformbool:
          #if np.random.rand(1) < 0.5:
          flag = True

          #try random noise by default, 30% chance gaussian, 30% normal, 40% speckle?
          #if uni_noise < 0.5:
          noise_array = np.random.uniform(0.95,1.05,256*256)
          noise_array.resize((256,256))
          
          vol = np.multiply(vol, noise_array)
          vol = np.clip(vol, 0, 255)
          vol = vol.astype(np.float32)
          
          self.transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.RandomHorizontalFlip(p=(hor_flip < 0.5)), 
            torchvision.transforms.RandomAffine((randomangle,randomangle), translate=(trans,trans), resample=PIL.Image.BILINEAR),
            torchvision.transforms.RandomPerspective(distortion_scale=0.3, p=(persp < 0.3)),
            torchvision.transforms.ToTensor(),
            #torchvision.transforms.RandomErasing(p=(rand_erase < 0.2),scale=(0.02, 0.33), ratio=(0.3, 3.3))
        ])

        if flag:
          for sliceindex in range(vol.shape[0]):
            vol[sliceindex] = self.transforms(np.array(vol[sliceindex]))

        vol = np.stack((vol,)*3, axis=1)
        vol_tensor = torch.FloatTensor(vol)
        label_tensor = torch.FloatTensor([self.labels[index]])

        return vol_tensor, label_tensor

    def __len__(self):
        return len(self.paths)

def load_data(diagnosis, orientation, transformbool, use_gpu=True):

    print('load_data', diagnosis, orientation)

    train_path = "/content/gdrive/My Drive/thesis/Data/train"
    valid_path = "/content/gdrive/My Drive/thesis/Data/valid"
    test_path = "/content/gdrive/My Drive/thesis/Data/test"

    batchsize = 1
    numworkers = 8
    
    #assert(1==2)
    #train_dataset = Dataset(train_dirs, diagnosis, use_gpu)
    train_dataset = Dataset(train_path, diagnosis, orientation, use_gpu, transformbool)
    valid_dataset = Dataset(valid_path, diagnosis, orientation, use_gpu, False)
    test_dataset = Dataset(test_path, diagnosis, orientation, use_gpu, False)

    train_loader = data.DataLoader(train_dataset, batch_size=batchsize, num_workers=numworkers, shuffle=True)
    valid_loader = data.DataLoader(valid_dataset, batch_size=batchsize, num_workers=numworkers, shuffle=False)
    test_loader = data.DataLoader(test_dataset, batch_size=batchsize, num_workers=numworkers, shuffle=False)
    return train_loader, valid_loader, test_loader


In [0]:
import matplotlib
matplotlib.use('Agg')
gpu = True
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

#learningrate = 5e-05
epochs = 100
#diagnosis = 'ACL'
rundir = "/content/gdrive/My Drive/thesis/Data"
#orientation = 'axial'
savedir = "/content/gdrive/My Drive/thesis/Results/round3"


if gpu:
  torch.cuda.manual_seed_all(seed)

def display_single(x_length, lr1, varray, tarray, testarray, title, xlabel, ylabel, save_dir):
  plt.figure(0)
  plt.title(title)
  plt.ylim(0.0,1.0)
  plt.plot(np.arange(x_length), varray, label='valid')
  plt.plot(np.arange(x_length), tarray, label='train')
  plt.plot(np.arange(x_length), testarray, label='test')
  plt.legend()
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.savefig(save_dir + '/' + title + '.eps', format='eps')
  plt.show()
  plt.close()
  return

In [0]:
#ax w/ aug
aug = True
epochs = 200
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-02
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000011
starting epoch 2. time passed: 0:03:22.139422
starting epoch 3. time passed: 0:04:34.673260
starting epoch 4. time passed: 0:05:50.109420
starting epoch 5. time passed: 0:07:06.322362
starting epoch 6. time passed: 0:08:20.115570
starting epoch 7. time passed: 0:09:34.365472
starting epoch 8. time passed: 0:10:46.264451
starting epoch 9. time passed: 0:11:58.084612
starting epoch 10. time passed: 0:13:10.220347
starting epoch 11. time passed: 0:14:23.896411
starting epoch 12. time passed: 0:15:37.906935
starting epoch 13. time passed: 0:16:52.062751
starting epoch 14. time passed: 0:18:05.555959
starting epoch 15. time passed: 0:19:19.620215
starting epoch 16. time passed: 0:20:33.502681
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.5000_train0.4898_test0.5000_epoch200


In [0]:
#ax w/ aug
aug = True
epochs = 100
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-03
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000008
starting epoch 2. time passed: 0:04:19.872767
starting epoch 3. time passed: 0:05:31.795957
starting epoch 4. time passed: 0:06:44.065455
starting epoch 5. time passed: 0:07:56.463251
starting epoch 6. time passed: 0:09:09.104236
starting epoch 7. time passed: 0:10:22.269240
starting epoch 8. time passed: 0:11:34.716854
starting epoch 9. time passed: 0:12:45.515556
starting epoch 10. time passed: 0:13:56.304909
starting epoch 11. time passed: 0:15:06.944933
starting epoch 12. time passed: 0:16:18.356741
starting epoch 13. time passed: 0:17:29.523407
starting epoch 14. time passed: 0:18:40.846314
starting epoch 15. time passed: 0:19:51.728981
starting epoch 16. time passed: 0:21:02.957856
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.6134_train0.4629_test0.6048_epoch100


In [0]:
#ax w/ aug
aug = True
epochs = 100
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-04
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000012
starting epoch 2. time passed: 0:01:10.626613
starting epoch 3. time passed: 0:02:22.783386
starting epoch 4. time passed: 0:03:36.396212
starting epoch 5. time passed: 0:04:50.644998
starting epoch 6. time passed: 0:06:04.537186
starting epoch 7. time passed: 0:07:18.270358
starting epoch 8. time passed: 0:08:30.764612
starting epoch 9. time passed: 0:09:42.019069
starting epoch 10. time passed: 0:10:53.166956
starting epoch 11. time passed: 0:12:04.317727
starting epoch 12. time passed: 0:13:15.094400
starting epoch 13. time passed: 0:14:25.828466
starting epoch 14. time passed: 0:15:37.164803
starting epoch 15. time passed: 0:16:48.079155
starting epoch 16. time passed: 0:17:59.241366
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.4739_train0.9597_test0.3764_epoch100


In [9]:
#ax w/ aug
aug = True
epochs = 100
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000013
starting epoch 2. time passed: 0:02:40.434871
starting epoch 3. time passed: 0:04:04.826383
starting epoch 4. time passed: 0:05:30.284295
starting epoch 5. time passed: 0:06:56.680092
starting epoch 6. time passed: 0:08:21.636569
starting epoch 7. time passed: 0:09:47.359755
starting epoch 8. time passed: 0:11:12.467351
starting epoch 9. time passed: 0:12:37.393218
starting epoch 10. time passed: 0:14:02.162331
starting epoch 11. time passed: 0:15:27.267405
starting epoch 12. time passed: 0:16:53.116567
starting epoch 13. time passed: 0:18:18.698843
starting epoch 14. time passed: 0:19:43.454368
starting epoch 15. time passed: 0:21:08.886763
starting epoch 16. time passed: 0:22:33.620315
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.7666_train0.8772_test0.7168_epoch100


In [10]:
#ax w/ aug
aug = True
epochs = 100
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-06
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000012
starting epoch 2. time passed: 0:01:22.417603
starting epoch 3. time passed: 0:02:46.636984
starting epoch 4. time passed: 0:04:10.969700
starting epoch 5. time passed: 0:05:36.029732
starting epoch 6. time passed: 0:07:00.712575
starting epoch 7. time passed: 0:08:25.714744
starting epoch 8. time passed: 0:09:51.125492
starting epoch 9. time passed: 0:11:16.401317
starting epoch 10. time passed: 0:12:42.379375
starting epoch 11. time passed: 0:14:08.106243
starting epoch 12. time passed: 0:15:33.370638
starting epoch 13. time passed: 0:16:58.922192
starting epoch 14. time passed: 0:18:24.782060
starting epoch 15. time passed: 0:19:50.577492
starting epoch 16. time passed: 0:21:16.683657
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.8479_train0.7538_test0.6982_epoch100


In [11]:
#ax w/ aug
aug = True
epochs = 100
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-07
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000012
starting epoch 2. time passed: 0:01:22.634551
starting epoch 3. time passed: 0:02:46.203513
starting epoch 4. time passed: 0:04:09.559447
starting epoch 5. time passed: 0:05:33.681507
starting epoch 6. time passed: 0:06:57.354202
starting epoch 7. time passed: 0:08:20.851592
starting epoch 8. time passed: 0:09:44.161360
starting epoch 9. time passed: 0:11:08.366363
starting epoch 10. time passed: 0:12:32.052625
starting epoch 11. time passed: 0:13:56.203124
starting epoch 12. time passed: 0:15:20.243529
starting epoch 13. time passed: 0:16:43.273097
starting epoch 14. time passed: 0:18:06.563474
starting epoch 15. time passed: 0:19:30.128442
starting epoch 16. time passed: 0:20:53.504622
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.8622_train0.7522_test0.6500_epoch100


In [12]:
#ax w/ aug
aug = True
epochs = 100
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-08
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

load_data ACL axial
/content/gdrive/My Drive/thesis/Data/train
(1000,)
[0.188, 0.812]
/content/gdrive/My Drive/thesis/Data/valid
(120,)
[0.45, 0.55]
/content/gdrive/My Drive/thesis/Data/test
(130,)
[0.15384615384615385, 0.8461538461538461]
starting epoch 1. time passed: 0:00:00.000010
starting epoch 2. time passed: 0:01:24.995420
starting epoch 3. time passed: 0:02:50.256284
starting epoch 4. time passed: 0:04:16.307599
starting epoch 5. time passed: 0:05:42.302851
starting epoch 6. time passed: 0:07:08.765626
starting epoch 7. time passed: 0:08:35.472111
starting epoch 8. time passed: 0:10:01.005906
starting epoch 9. time passed: 0:11:26.566486
starting epoch 10. time passed: 0:12:51.635032
starting epoch 11. time passed: 0:14:17.330831
starting epoch 12. time passed: 0:15:42.992099
starting epoch 13. time passed: 0:17:09.311158
starting epoch 14. time passed: 0:18:35.998443
starting epoch 15. time passed: 0:20:01.843085
starting epoch 16. time passed: 0:21:27.935265
starting epoch 17

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


model saved at /content/gdrive/My Drive/thesis/Results/models/ACL/axial/val0.6263_train0.5500_test0.5277_epoch100


In [0]:
#ax w/ aug
aug = True
epochs = 50
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

In [0]:
#ax w/ aug
aug = True
epochs = 50
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

In [0]:
#ax w/ aug
aug = True
epochs = 50
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

In [0]:
#ax w/ aug
aug = True
epochs = 50
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)

In [0]:
#ax w/ aug
aug = True
epochs = 50
diagnosis = 'ACL'
orientation = 'axial'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet untrained RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)